## Week 2: Refrigeration

Welcome to week two of our python heat transfer tutorials. This week we're going to be looking at another one of your homework questions from last week; looking at refrigeration cycles.

![Alt text](ts.png)

1.10. A refrigerator uses refrigerant-134a as the working fluid and operates on an ideal vapor-compression refrigeration cycle between 0.14 and 0.8 MPa. If the mass flow rate of the refrigerant is 0.05 kg/s, determine:
(a) the rate of heat removal from the refrigerated space and the power input to the compressor, 
(b) the rate of heat rejection to the environment, and 
(c) the COP of the refrigerator.

The first thing that we're going to do is import numpy, a handy matrix based python class, kind of like matlab. If you're interested, you can read more about numpy [here](http://www.numpy.org/)

In [2]:
import numpy as np

Now we're going to use numpy to import the thermal properties table from the csv file you downloaded with this tutorial, so that we can manipulate it and make it do our bidding

In [18]:
fridgetab = np.genfromtxt('thermoproperties.csv', dtype=None, delimiter = ",")

This imports the data as an numpy array. Numpy arrays are useful in this particular instance because we can call the sections of the data that we are interested in. Note that in numpy arrays indexing starts from 0

In [19]:
fridgetab[0:3, :]

array([[b'R134a - TetraFlouroEthane Saturation Properties - Pressure Table',
        b'', b'', b'', b'', b'', b'', b'', b'', b''],
       [b'Pressure', b'Temp', b'volume (m^3/kg)', b'', b'enthalpy (kJ/kg)',
        b'', b'', b'entropy (kJ/kg.K)', b'', b''],
       [b'kPa', b'\xc2\xb0C', b'vf', b'vg', b'hf', b'hfg', b'hg', b'sf',
        b'sfg', b'sg']], 
      dtype='|S64')

Now we can call the data necessary for our question. So We know that we need the data for 0.14 and 0.8 MPa, how do we call it? we need to figure out where it sits in our numpy array. We know that the data in this case starts from the fifth row at 80 Pa:

In [20]:
fridgetab[4,:]

array([b'80', b'-31.1', b'0.0007185', b'0.2376', b'11.3', b'220.2',
       b'231.5', b'0.0472', b'0.91', b'0.9572'], 
      dtype='|S64')

and increases in increments of 20 Pa:

In [21]:
fridgetab[5,:]

array([b'100', b'-26.4', b'0.0007259', b'0.1926', b'17.3', b'217.2',
       b'234.5', b'0.072', b'0.8799', b'0.9519'], 
      dtype='|S64')

So we want $h_{1-4}$ and $s_{1-2}$:

$h_1 = h_{g@0.14MPa}$

$s_1 = s_{g@0.14MPa}$

$h_2 = h_{g@0.8MPa}$

$s_2 = s_1$

$h_3 = h_{f@0.8MPa}$

$h_4 \cong h_3$ (why $\cong$ and not = ?)

So we need the data for 0.8 and 0.14 MPa respectively, working out that their indices will correspond to $i = \frac{Pr-0.08}{0.02} + 3$ for $Pr \leq 360$, $i = \frac{Pr-0.4}{0.1} + 21$ for $400 \leq Pr \leq 1000$ and $i = \frac{Pr-1}{0.2} + 27$  for $1000 \leq Pr$

In [51]:
fridgetab[round((0.14 - 0.08)/0.02)+4, :]

array([b'140', b'-18.8', b'0.0007383', b'0.1402', b'27.1', b'212.1',
       b'239.2', b'0.111', b'0.8337', b'0.9446'], 
      dtype='|S64')

We can check which collumn has what data from the second and third row:

In [56]:
fridgetab[1:3, :]

array([[b'Pressure', b'Temp', b'volume (m^3/kg)', b'', b'enthalpy (kJ/kg)',
        b'', b'', b'entropy (kJ/kg.K)', b'', b''],
       [b'kPa', b'\xc2\xb0C', b'vf', b'vg', b'hf', b'hfg', b'hg', b'sf',
        b'sfg', b'sg']], 
      dtype='|S64')

The command np.where can be used to find the index for the specific elements we are trying to call, which can then be defined as variables:

In [57]:
hg = np.where(fridgetab==b'hg')[1][0]
hf = np.where(fridgetab==b'hf')[1][0]
sg = np.where(fridgetab==b'sg')[1][0]

Let's define a function that calls the data for a given Pressure, we can do this using if loops to call the appropriate algorithm for a given pressure. You should remember if loops from matlab. Below is an example of if loop syntax in python (Remember, indentation matters!) 

In [58]:
a = 'frog'
b = 'frog'
if a == b:
    print('ribbit')

ribbit


Coming back to the question at hand

(a) the rate of heat removal from the refrigerated space and the power input to the compressor

we recall the definition for the rate of heat removal:

$$\dot{Q}_L = \dot{m}(h_1 - h_4)$$

4